In [117]:
import pandas as pd
import numpy as np
import os, sys
import geopandas as gpd
import geopy

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import seaborn as sns

In [118]:
pd.set_option('display.max_rows', 500000)
pd.set_option('display.max_columns', 50000)

In [119]:
Df_corrigidos = pd.read_excel(r'C:\Doutorado\BD\SESPA\Dados_adquiridos_em_04_10_2018\Dados_Corrigidos\Dados_adquiridos_em_04_10_2018_corrigidos.xlsx')

Df_corrigidos.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,NU_ANO,CLASSI_FIN,CLAS_ETIOL,ID_MN_RESI,ID_RG_RESI,ID_BAIRRO,NM_BAIRRO,NM_LOGRADO,NU_NUMERO,Logradouro_estimado_por_Philipe,Coordenada_estimada_manualmente,NM_COMPLEM,NM_REFEREN,NU_CEP
0,118279,2008-09-05,200836,2008,1,1,150580,1491,987.0,CIDADE NOVA,01 DE MAIO,402,01 DE MAIO,NaN,NaN,NaN,68480000
1,267317,2010-04-29,201017,2010,1,1,150830,1487,NaN,CIDADE NOVA,03 DE MAIO,NaN,03 DE MAIO,NaN,NaN,NaN,68620000
2,543155,2012-10-18,201242,2012,1,1,150830,1487,NaN,MANGUEIRAO,08 DE MAIO,61,08 DE MAIO,NaN,NaN,NaN,NaN
3,140744,2008-02-07,200806,2008,1,1,150442,1484,404.0,DECOUVILLE,09ª TRAV.,11,09ª TRAV.,NaN,NaN,SERRARIA,NaN
4,316052,2011-08-01,201131,2011,1,1,150580,1491,984.0,BOSQUE,1 DE MAIO,104,1 DE MAIO,NaN,NaN,NaN,68480000


In [120]:
def replace_strings(x, to_replace='standard'):
    
    if to_replace.startswith('stand'):
        to_replace = {', SN':'',
                     '15*': '15',
                      ', nan':'',
                      '- nan':'',
                      '...': ''}
    
    x = str(x)
    
    for key, value in to_replace.items():
    
        x = x.replace(key, value)

    return x

def get_full_geocoding_name(x):
    path = ('Brazil, Pará, '  
             + str(x['Logradouro_estimado_por_Philipe']) + ', ' 
             + str(x['NU_NUMERO']) + ', '
             + str(x['NU_CEP']))
        
    return replace_strings(path)



Df_corrigidos = Df_corrigidos.head(100)

Df_corrigidos['full_name'] = Df_corrigidos.apply(get_full_geocoding_name, axis=1)


In [128]:
Df_corrigidos.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,NU_ANO,CLASSI_FIN,CLAS_ETIOL,ID_MN_RESI,ID_RG_RESI,ID_BAIRRO,NM_BAIRRO,NM_LOGRADO,NU_NUMERO,Logradouro_estimado_por_Philipe,Coordenada_estimada_manualmente,NM_COMPLEM,NM_REFEREN,NU_CEP,full_name
0,118279,2008-09-05,200836,2008,1,1,150580,1491,987.0,CIDADE NOVA,01 DE MAIO,402,01 DE MAIO,NaN,NaN,NaN,68480000,"Brazil, Pará, 01 DE MAIO, 402, 68480000"
1,267317,2010-04-29,201017,2010,1,1,150830,1487,NaN,CIDADE NOVA,03 DE MAIO,NaN,03 DE MAIO,NaN,NaN,NaN,68620000,"Brazil, Pará, 03 DE MAIO, 68620000"
2,543155,2012-10-18,201242,2012,1,1,150830,1487,NaN,MANGUEIRAO,08 DE MAIO,61,08 DE MAIO,NaN,NaN,NaN,NaN,"Brazil, Pará, 08 DE MAIO, 61"
3,140744,2008-02-07,200806,2008,1,1,150442,1484,404.0,DECOUVILLE,09ª TRAV.,11,09ª TRAV.,NaN,NaN,SERRARIA,NaN,"Brazil, Pará, 09ª TRAV., 11"
4,316052,2011-08-01,201131,2011,1,1,150580,1491,984.0,BOSQUE,1 DE MAIO,104,1 DE MAIO,NaN,NaN,NaN,68480000,"Brazil, Pará, 1 DE MAIO, 104, 68480000"


In [139]:
from shapely.geometry import Point
from geopy.extra.rate_limiter import RateLimiter
import warnings

In [ ]:
def geocode_OpenMapQuest(name, 
                         country_codes=None, 
                         API_Key = 'arjIxMIcVzqDJx7fYrv854lrGLAj5geM',
                         **kwargs):
    
    min_delay_seconds = kwargs.pop('min_delay_seconds', 5)
    
    
    exactly_one= kwargs.pop('exactly_one', True)
    
    OPenMapQuest_Geocoder = geopy.geocoders.OpenMapQuest(api_key=API_Key, 
                                                         country_bias=country_codes
                                                        , **kwargs)
    
    geocode = RateLimiter(OPenMapQuest_Geocoder.geocode,
                          min_delay_seconds=min_delay_seconds)
    warnings.warnings.catch_warnings():
        warnings.filterwarnings(action='ignore')
        
        geocoded = geocode(query=name, 
                               country_codes=country_codes,
                               exactly_one=exactly_one)
    
    if geocoded is None:
        return np.nan
    
    else:
        Points = []
        
        if isinstance(geocoded, list):

            for geocode in geocoded:
                location, point = geocode
                P = Point(reversed(point))
                Points.append(P)

            return Points
        
        else:
            location, point = geocoded
            return [Point(reversed(point))]
            

def geocode_via_GeoLake(name, 
                        country_codes=None, 
                        api_key='ykbklV3MvgAMu3tLJQ7f',
                        **kwargs):
    
    min_delay_seconds = kwargs.pop('min_delay_seconds', 5)
    exactly_one= kwargs.pop('exactly_one', True)
    
    Geolake_Geocoder = geopy.geocoders.Geolake(api_key=api_key, 
                                               **kwargs)
    
    geocode = RateLimiter(Geolake_Geocoder.geocode, 
                          min_delay_seconds=min_delay_seconds)
    
    warnings.warnings.catch_warnings():
        warnings.filterwarnings(action='ignore')
        
        geocoded = geocode(query=name, country_codes=country_codes,
                               exactly_one=exactly_one)
    
    if geocoded is None:
        return np.nan
    
    else:
        Points = []
        
        if isinstance(geocoded, list):

            for geocode in geocoded:
                location, point = geocode
                P = Point(reversed(point))
                Points.append(P)

            return Points
        
        else:
            location, point = geocoded
            return [Point(reversed(point))]
    
def get_point(R1, R2):
    if np.isnan(R1) and isinstance(R2, list):
        return R2
    
    elif isinstance(R1, list) and np.isnan(R2):
        return R1
    
    elif isinstance(R1, list) and isinstance(R2, list):
        return R1
    
    else:
        R = np.nan
        
    if isinstance(R, str):
        R = np.nan
    return R

def geocoding(name, 
              country_codes='BR', min_delay_seconds=5, 
              **kwargs):
    
    api_key_GeoLake=kwargs.pop('api_key_GeoLake'),
    api_key_OpenMapQuest=kwargs.pop('api_key_OpenMapQuest'),
    
    R1 = geocode_via_GeoLake(name, country_codes, 
                             api_key_GeoLake,
                             min_delay_seconds=min_delay_seconds, 
                             
                             **kwargs)
    
    R2 = geocode_OpenMapQuest(name, country_codes, 
                              api_key_OpenMapQuest,
                              min_delay_seconds=min_delay_seconds, 
                              
                              **kwargs)
    
    return get_point(R1, R2)

from tqdm import tqdm
tqdm.pandas()

def df_geocoder(df, api_key_GeoLake,
                api_key_OpenMapQuest,
                logradouro='full_name',
                exactly_one=False,
                country_codes='BR', 
                min_delay_seconds=5):
    
    df = df.copy()
    df['geocoded'] = df[logradouro].progress_apply(lambda x: geocoding(x, 
                                                                       api_key_GeoLake=api_key_GeoLake,
                                                                       api_key_OpenMapQuest=api_key_OpenMapQuest,
                                                                       country_codes=country_codes,
                                                                       min_delay_seconds=5,
                                                                       exactly_one=exactly_one))

    r = (df.set_index([logradouro])['geocoded'].apply(pd.Series)
         .stack().reset_index()
         .drop({'level_1'}, axis=1)
        ).copy()
    
    r.columns = [logradouro, 'geometry']

    return gpd.GeoDataFrame(r.merge(df, on=logradouro), crs={'init':'epsg:4326'}).drop({'geocoded'}, axis=1)   

In [146]:
GDf_corrigidos = df_geocoder(Df_corrigidos.iloc[44:46], 
                             api_key_GeoLake ='ykbklV3MvgAMu3tLJQ7f',
                             api_key_OpenMapQuest='arjIxMIcVzqDJx7fYrv854lrGLAj5geM',
                             logradouro='full_name',
                             exactly_one=False,
                             country_codes='BR', 
                             min_delay_seconds=5)

GDf_corrigidos













  0%|          | 0/2 [00:00<?, ?it/s]RateLimiter caught an error, retrying (0/2 tries). Called with (*(), **{'query': 'Brazil, Pará, 6° RUA, 68480000', 'country_codes': 'BR', 'exactly_one': False}).
Traceback (most recent call last):
  File "C:\Anaconda3\envs\Python_3.7\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.

RateLimiter swallowed an error after 2 retries. Called with (*(), **{'query': 'Brazil, Pará, 6° RUA, 68480000', 'country_codes': 'BR', 'exactly_one': False}).
Traceback (most recent call last):
  File "C:\Anaconda3\envs\Python_3.7\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPErr

RateLimiter caught an error, retrying (1/2 tries). Called with (*(), **{'query': 'Brazil, Pará, 6ª RUA, 68635000', 'country_codes': 'BR', 'exactly_one': False}).
Traceback (most recent call last):
  File "C:\Anaconda3\envs\Python_3.7\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Anaconda3\envs\Python_3.7\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTP

,geometry,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,NU_ANO,CLASSI_FIN,CLAS_ETIOL,ID_MN_RESI,ID_RG_RESI,ID_BAIRRO,NM_BAIRRO,NM_LOGRADO,NU_NUMERO,Logradouro_estimado_por_Philipe,Coordenada_estimada_manualmente,NM_COMPLEM,NM_REFEREN,NU_CEP,full_name


In [129]:
%matplotlib
import cartopy
import cartopy.crs as ccrs

Using matplotlib backend: Qt5Agg


In [132]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines(resolution='50m');

from matplotlib import ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = False
gl.xlines = False
gl.xlocator = mticker.FixedLocator([-180, -45, 0, 45, 180])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 15, 'color': 'gray'}
gl.xlabel_style = {'color': 'red', 'weight': 'bold'}

GDf_corrigidos.plot(ax=ax, cmap='viridis', categorical=True, column='full_name', transform=ccrs.Geodetic())

In [131]:
plt.figure()
GDf_corrigidos.plot(cmap='viridis', categorical=True, column='full_name')